In [ ]:
import numpy as np
import pandas as pd
import warnings
import json
import joblib
import time
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import optuna

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
np.random.seed(42)

# =============================================================================
# CONFIGURATION
# =============================================================================
print("="*80)
print("GRADIENT BOOSTING HYPERPARAMETER OPTIMIZATION")
print("="*80)

# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../data/preprocessed/X_train.pkl')
y_train = joblib.load('../data/preprocessed/y_train.pkl')
metadata = joblib.load('../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Target shape: {y_train.shape}")

# =============================================================================
# OPTIMIZATION SETTINGS
# =============================================================================
FAST_MODE = True  # Set to False for more thorough search
N_TRIALS = 5 if FAST_MODE else 20
N_FOLDS = 4

print(f"\n{'='*60}")
print(f"OPTIMIZATION SETTINGS")
print(f"{'='*60}")
print(f"Mode: {'FAST' if FAST_MODE else 'THOROUGH'}")
print(f"Trials: {N_TRIALS}")
print(f"Folds: {N_FOLDS}")

# =============================================================================
# OBJECTIVE FUNCTION
# =============================================================================
def objective_gbr(trial, X, y):
    """Objective function for GradientBoosting optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_samples_split': trial.suggest_int('min_samples_split', 10, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'max_features': trial.suggest_float('max_features', 0.6, 0.9),
        'random_state': 42
    }

    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        model = GradientBoostingRegressor(**params)
        model.fit(X_tr, y_tr)

        preds = np.clip(model.predict(X_val), 0, 100)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(rmse)

    return np.mean(scores)

# =============================================================================
# RUN OPTIMIZATION
# =============================================================================
print("\n" + "="*80)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*80)

start_time = time.time()

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(
    lambda trial: objective_gbr(trial, X_train, y_train),
    n_trials=N_TRIALS,
    show_progress_bar=True,
    n_jobs=4
)

optimization_time = time.time() - start_time

# =============================================================================
# RESULTS
# =============================================================================
print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(f"Best RMSE: {study.best_value:.6f}")
print(f"Optimization time: {optimization_time:.1f}s")
print(f"\nBest parameters:")
for param, value in study.best_params.items():
    print(f"  {param:20s}: {value}")

# =============================================================================
# SAVE RESULTS
# =============================================================================
import os
os.makedirs('../results/optimization', exist_ok=True)

# Save best parameters
best_params = study.best_params.copy()
best_params.update({
    'random_state': 42
})

joblib.dump(best_params, '../results/optimization/gradientboosting_params.pkl')
print("\n✓ Parameters saved to: ../results/optimization/gradientboosting_params.pkl")

# Save optimization history
history = pd.DataFrame({
    'trial': [t.number for t in study.trials],
    'value': [t.value for t in study.trials],
    'params': [str(t.params) for t in study.trials]
})
history.to_csv('../results/optimization/gradientboosting_history.csv', index=False)
print("✓ History saved to: ../results/optimization/gradientboosting_history.csv")

# Save study object
joblib.dump(study, '../results/optimization/gradientboosting_study.pkl')
print("✓ Study saved to: ../results/optimization/gradientboosting_study.pkl")

# Save summary
summary = {
    'model': 'GradientBoosting',
    'best_rmse': study.best_value,
    'n_trials': N_TRIALS,
    'n_folds': N_FOLDS,
    'optimization_time': optimization_time,
    'best_params': best_params
}
joblib.dump(summary, '../results/optimization/gradientboosting_summary.pkl')
print("✓ Summary saved to: ../results/optimization/gradientboosting_summary.pkl")

print("\n" + "="*80)
print("GRADIENT BOOSTING OPTIMIZATION COMPLETE!")
print("="*80)
print(f"✓ Best CV RMSE: {study.best_value:.6f}")
print(f"✓ All results saved in '../results/optimization/'")